In [90]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re as re

%matplotlib inline

In [91]:
# loading the dataset
train = pd.read_csv('train.csv', index_col='PassengerId')
test = pd.read_csv('test.csv', index_col='PassengerId')
dataset_list = [train, test] # to do feature engineering and data cleaning
train.shape, test.shape

((891, 11), (418, 10))

In [92]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [93]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [94]:
train.describe(exclude='number')


,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Garside, Miss. Ethel",male,347082,C23 C25 C27,S
freq,1,577,7,4,644


In [95]:
train.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [96]:
test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [97]:
train.groupby('Embarked')[['Survived']].mean()

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [98]:
train.pivot_table(index='Sex', columns='Pclass', values='Survived')


Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [99]:
# to get pivot_table for (Sex, Age) need to discretize the Age
age_bin = pd.cut(train['Age'], bins=[0, 18, 80], labels=['underaged', 'adult'])
train.pivot_table(index='Sex', columns=age_bin, values='Survived')

Age,nan,adult
Sex,,
female,0.782383,0.676471
male,0.180628,0.338028


In [100]:
y_train = train['Survived']
train = train.drop('Survived', axis=1)
train['Training_set'] = True
test['Training_set'] = False
df_comb = pd.concat([train, test])
df_comb.isnull().sum()

Pclass             0
Name               0
Sex                0
Age              263
SibSp              0
Parch              0
Ticket             0
Fare               1
Cabin           1014
Embarked           2
Training_set       0
dtype: int64

In [101]:
df_comb[df_comb.Embarked.isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Training_set
PassengerId,,,,,,,,,,,
62,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,True
830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,True


In [102]:
df_comb[df_comb['Fare'].between(75, 85) & (df_comb.Pclass==1) & (df_comb.Sex=='female') & (df_comb.Cabin.str.startswith('B'))]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Training_set
PassengerId,,,,,,,,,,,
62,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0000,B28,NaN,True
830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0000,B28,NaN,True
904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S,False
1289,1,"Frolicher-Stehli, Mrs. Maxmillian (Margaretha ...",female,48.0,1,1,13567,79.2000,B41,C,False


In [103]:
df_comb['Embarked']= df_comb['Embarked'].fillna(value=df_comb['Embarked'].value_counts().index[0])
df_comb.loc[[62, 830]]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Training_set
PassengerId,,,,,,,,,,,
62,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S,True
830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,S,True


In [104]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy='median')
imputer.fit(df_comb[['Age', 'Fare']])
df_comb[['Age', 'Fare']] = imputer.transform(df_comb[['Age', 'Fare']])

In [106]:
# Title derived from Name
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name) # regex starting with space ending with .
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

df_comb['Title'] = df_comb['Name'].apply(get_title)

print(pd.crosstab(df_comb['Title'], train['Sex']))

Sex       female  male
Title                 
Capt           0     1
Col            0     2
Countess       1     0
Don            0     1
Dr             1     6
Jonkheer       0     1
Lady           1     0
Major          0     2
Master         0    40
Miss         182     0
Mlle           2     0
Mme            1     0
Mr             0   517
Mrs          125     0
Ms             1     0
Rev            0     6
Sir            0     1


In [107]:
# 5. Title Group all non-common titles into group based on gender
df_comb['Title'] = df_comb['Title'].replace(['Mlle','Ms', 'Countess', 'Lady', 'Dona'], 'Miss')
df_comb['Title'] = df_comb['Title'].replace('Mme', 'Mrs')
df_comb['Title'] = df_comb['Title'].replace(['Capt','Col', 'Don', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Mr')

print(pd.crosstab(df_comb['Title'], df_comb['Sex']))

Sex     female  male
Title               
Dr           1     7
Master       0    61
Miss       267     0
Mr           0   775
Mrs        198     0


In [108]:
# 2. IsAlone
# check if columns have duplicate ticket number. Duplicate ticket numbers symbolize family/friends
tks = df_comb['Ticket']
dataset_dup = df_comb[tks.isin(tks[tks.duplicated()])]

In [109]:
df_comb['TkDup'] = df_comb.Ticket.isin(dataset_dup['Ticket'])

In [110]:
df_comb['IsAlone'] = 0
df_comb['FamilySize'] = df_comb['SibSp'] + df_comb['Parch'] + 1
dataset_filter = (df_comb['FamilySize'] == 1) & (df_comb['TkDup'] == False)
df_comb['IsAlone'] = np.where(dataset_filter, 1, 0)

In [111]:
df_comb.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Training_set,Title,TkDup,IsAlone,FamilySize
PassengerId,,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,Mr,False,0,2
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True,Mrs,True,0,2
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,Miss,False,1,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True,Mrs,True,0,2
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,Mr,False,1,1


In [112]:
# OneHotEncoder for Sex, Embarked:
df_comb = df_comb.join(pd.get_dummies(df_comb[['Sex', 'Embarked','Title']]))

In [113]:
df_comb.drop(['Sex','Embarked','Title'], axis=1, inplace=True)
df_comb.tail()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Training_set,TkDup,...,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Dr,Title_Master,Title_Miss,Title_Mr,Title_Mrs
PassengerId,,,,,,,,,,,,,,,,,,,,,
1305,3,"Spector, Mr. Woolf",28.0,0,0,A.5. 3236,8.0500,NaN,False,False,...,0,1,0,0,1,0,0,0,1,0
1306,1,"Oliva y Ocana, Dona. Fermina",39.0,0,0,PC 17758,108.9000,C105,False,True,...,1,0,1,0,0,0,0,1,0,0
1307,3,"Saether, Mr. Simon Sivertsen",38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,False,False,...,0,1,0,0,1,0,0,0,1,0
1308,3,"Ware, Mr. Frederick",28.0,0,0,359309,8.0500,NaN,False,False,...,0,1,0,0,1,0,0,0,1,0
1309,3,"Peter, Master. Michael J",28.0,1,1,2668,22.3583,NaN,False,True,...,0,1,1,0,0,0,1,0,0,0


In [117]:
df_comb.loc[df_comb.Cabin.notnull() & df_comb.Cabin.str.contains('F'), 'Cabin']

PassengerId
67        F33
76      F G73
129     F E69
149        F2
184        F4
194        F2
341        F2
346       F33
517       F33
619        F4
700     F G63
716     F G73
777       F38
949     F G63
1001        F
1070       F4
1114      F33
1180    F E46
1213    F E57
1214       F2
1218       F4
Name: Cabin, dtype: object

In [118]:
deck_list = list(map(lambda x: x[0], df_comb.Cabin.dropna().tolist()))
print(deck_list)

['C', 'C', 'E', 'G', 'C', 'D', 'A', 'C', 'B', 'D', 'B', 'C', 'B', 'C', 'F', 'F', 'C', 'E', 'A', 'D', 'D', 'C', 'B', 'E', 'D', 'F', 'D', 'C', 'B', 'F', 'C', 'E', 'B', 'A', 'C', 'F', 'A', 'F', 'B', 'B', 'G', 'A', 'D', 'D', 'C', 'C', 'C', 'D', 'G', 'C', 'B', 'E', 'B', 'C', 'C', 'C', 'D', 'A', 'B', 'D', 'C', 'C', 'B', 'E', 'C', 'C', 'E', 'C', 'B', 'C', 'E', 'C', 'D', 'B', 'C', 'C', 'C', 'E', 'T', 'F', 'C', 'F', 'C', 'E', 'D', 'B', 'E', 'C', 'B', 'D', 'G', 'C', 'E', 'C', 'E', 'B', 'C', 'A', 'C', 'C', 'C', 'E', 'D', 'E', 'E', 'D', 'A', 'B', 'C', 'B', 'C', 'D', 'C', 'B', 'C', 'E', 'D', 'F', 'B', 'B', 'C', 'B', 'B', 'B', 'C', 'C', 'A', 'E', 'C', 'E', 'E', 'C', 'A', 'E', 'B', 'D', 'A', 'C', 'F', 'D', 'D', 'D', 'A', 'B', 'B', 'D', 'A', 'D', 'E', 'C', 'B', 'B', 'D', 'B', 'B', 'C', 'F', 'C', 'E', 'E', 'C', 'C', 'C', 'F', 'C', 'E', 'E', 'B', 'B', 'D', 'C', 'B', 'B', 'D', 'E', 'B', 'B', 'D', 'E', 'F', 'B', 'B', 'D', 'B', 'D', 'B', 'A', 'E', 'B', 'B', 'E', 'B', 'E', 'C', 'C', 'D', 'E', 'D', 'A', 'D',

In [120]:
# Deck as the initial of the Cabin. If no cabin, use 'X'
crit = df_comb['Cabin'].isnull()
df_comb['Deck'] = df_comb['Cabin'].astype(str).str[0].where(~crit, other='X')
df_comb.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Training_set,TkDup,...,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Dr,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Deck
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,True,False,...,1,0,0,1,0,0,0,1,0,X
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,True,True,...,0,1,0,0,0,0,0,0,1,C
3,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,True,False,...,0,0,0,1,0,0,1,0,0,X
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,True,True,...,0,0,0,1,0,0,0,0,1,C
5,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,True,False,...,1,0,0,1,0,0,0,1,0,X


In [121]:
# some passengers booked more than a single cabin
df_comb.loc[[28, 76, 89, 129]]

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Training_set,TkDup,...,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Title_Dr,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Deck
PassengerId,,,,,,,,,,,,,,,,,,,,,
28,1,"Fortune, Mr. Charles Alexander",19.0,3,2,19950,263.0000,C23 C25 C27,True,True,...,1,0,0,1,0,0,0,1,0,C
76,3,"Moen, Mr. Sigurd Hansen",25.0,0,0,348123,7.6500,F G73,True,False,...,1,0,0,1,0,0,0,1,0,F
89,1,"Fortune, Miss. Mabel Helen",23.0,3,2,19950,263.0000,C23 C25 C27,True,True,...,0,0,0,1,0,0,1,0,0,C
129,3,"Peter, Miss. Anna",28.0,1,1,2668,22.3583,F E69,True,True,...,0,1,0,0,0,0,1,0,0,F


In [122]:
df_comb = df_comb.join(pd.get_dummies(df_comb[['Deck']]))
df_comb.drop("Deck", axis=1, inplace=True)
df_comb.tail()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Training_set,TkDup,...,Title_Mrs,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_X
PassengerId,,,,,,,,,,,,,,,,,,,,,
1305,3,"Spector, Mr. Woolf",28.0,0,0,A.5. 3236,8.0500,NaN,False,False,...,0,0,0,0,0,0,0,0,0,1
1306,1,"Oliva y Ocana, Dona. Fermina",39.0,0,0,PC 17758,108.9000,C105,False,True,...,0,0,0,1,0,0,0,0,0,0
1307,3,"Saether, Mr. Simon Sivertsen",38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,False,False,...,0,0,0,0,0,0,0,0,0,1
1308,3,"Ware, Mr. Frederick",28.0,0,0,359309,8.0500,NaN,False,False,...,0,0,0,0,0,0,0,0,0,1
1309,3,"Peter, Master. Michael J",28.0,1,1,2668,22.3583,NaN,False,True,...,0,0,0,0,0,0,0,0,0,1


In [123]:
train = df_comb[df_comb['Training_set'] == True]
train.drop('Training_set', axis=1)
test = df_comb[df_comb['Training_set'] == False]
test.drop('Training_set', axis=1)
train.shape, test.shape

((891, 31), (418, 31))

In [124]:
train.describe(include='number')


,Pclass,Age,SibSp,Parch,Fare,IsAlone,FamilySize,Sex_female,Sex_male,Embarked_C,...,Title_Mrs,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Deck_X
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.361582,0.523008,0.381594,32.204208,0.502806,1.904602,0.352413,0.647587,0.188552,...,0.141414,0.016835,0.052750,0.066218,0.037037,0.035915,0.014590,0.004489,0.001122,0.771044
std,0.836071,13.019697,1.102743,0.806057,49.693429,0.500273,1.613459,0.477990,0.477990,0.391372,...,0.348644,0.128725,0.223659,0.248802,0.188959,0.186182,0.119973,0.066890,0.033501,0.420397
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,3.000000,28.000000,0.000000,0.000000,14.454200,1.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,2.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,11.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [125]:
train.describe(include='O')


,Name,Ticket,Cabin
count,891,891,204
unique,891,681,147
top,"Garside, Miss. Ethel",347082,C23 C25 C27
freq,1,7,4


In [126]:
X_train = train.select_dtypes('number')
X_test = test.select_dtypes('number')
y_train.shape, X_train.shape, X_test.shape

((891,), (891, 26), (418, 26))

In [127]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_Pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log

83.5

In [128]:
df_lr_coeff = pd.DataFrame(X_train.columns)
df_lr_coeff.columns = ['Feature']
df_lr_coeff['Correlation'] = pd.Series(logreg.coef_[0])
df_lr_coeff.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
13,Title_Master,1.769057
7,Sex_female,1.397552
21,Deck_E,1.013732
16,Title_Mrs,0.965232
20,Deck_D,0.758960
9,Embarked_C,0.676876
10,Embarked_Q,0.500090
22,Deck_F,0.364798
14,Title_Miss,0.309487
11,Embarked_S,0.282037


In [129]:
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [130]:
# Estimators not based on the Decision Tree requires standardised/scaled data:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
scoring = []
models= [Perceptron(), SGDClassifier(), GaussianNB(), LogisticRegression(), \
         KNeighborsClassifier(n_neighbors=3), LinearSVC(), SVC(), DecisionTreeClassifier(),\
         RandomForestClassifier(n_estimators=100), AdaBoostClassifier(n_estimators=100),\
         GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=4), XGBClassifier()]
for model in models:
    ssmodel = make_pipeline(StandardScaler(), model)
    ssmodel.fit(X_train, y_train)
    y_Pred = ssmodel.predict(X_test)
    scoring.append(ssmodel.score(X_train, y_train) * 100)
    
df_scores = pd.DataFrame({"Model": models, "Score": scoring})
df_scores.sort_values(by='Score', ascending=False)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


,Model,Score
7,"DecisionTreeClassifier(class_weight=None, crit...",98.316498
10,([DecisionTreeRegressor(criterion='friedman_ms...,98.316498
8,"(DecisionTreeClassifier(class_weight=None, cri...",98.204265
11,"XGBClassifier(base_score=0.5, booster='gbtree'...",88.215488
4,"KNeighborsClassifier(algorithm='auto', leaf_si...",86.980920
9,"(DecisionTreeClassifier(class_weight=None, cri...",85.858586
6,"SVC(C=1.0, cache_size=200, class_weight=None, ...",85.409652
3,"LogisticRegression(C=1.0, class_weight=None, d...",83.613917
5,"LinearSVC(C=1.0, class_weight=None, dual=True,...",83.389450
0,"Perceptron(alpha=0.0001, class_weight=None, et...",80.134680


In [131]:
from sklearn.model_selection import RandomizedSearchCV
gbs = GradientBoostingClassifier()
grid = {'n_estimators': range(100, 300, 100),\
        'learning_rate': np.linspace(0.1, 1, 10),\
        'max_depth': np.arange(1, 10)}
rs = RandomizedSearchCV(estimator=gbs, param_distributions=grid, scoring='accuracy',\
                        n_iter=50, cv=5, n_jobs=-1)
rs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
          fit_params=None, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'n_estimators': range(100, 300, 100), 'learning_rate': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]), 'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [132]:
rs.best_score_, rs.best_params_


(0.82940516273849607,
 {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.10000000000000001})

In [136]:
# RandomForrestClassifier, 100 trees:
model = models[8]
model.fit(X_train, y_train)
y_Pred = model.predict(X_test)
acc = round(model.score(X_train, y_train) * 100, 2)
acc

98.319999999999993

In [134]:
from sklearn.model_selection import cross_val_score
cv_results = cross_val_score(model, X_train, y_train, cv=10, scoring='f1')
cv_results.mean()

0.74486180832258386

In [137]:
yT = model.predict(X_test)
my_1st_submission = pd.DataFrame({'PassengerId': test.index, 'Survived': yT})
my_1st_submission.to_csv('Titanic_RFC_01.csv', index=False)